In [1]:
import pandas as pd
import numpy as np
import networkx as nx

In [2]:
def getData(college):
    offers = pd.read_csv(f"data/offers/{college}_offers.csv")
    commits = pd.read_csv(f"data/commits/{college}_commits.csv")

    offers = offers.loc[offers['ranking'] != 0]
    commits = commits.loc[commits['ranking'] != 0]

    return offers, commits

In [3]:
def getWeightedReciprocity(college):
    offers, commits = getData(college)

    return 2 * np.sum(commits['ranking']) / (np.sum(commits['ranking']) + np.sum(offers['ranking'])) 

In [4]:
def getReciprocity(college):
    offers, commits = getData(college)

    return 2 * len(commits['ranking']) / (len(commits['ranking']) + len(offers['ranking'])) 

In [5]:
def getAvgRank(college):
    offers, commits = getData(college)

    return np.mean(offers['ranking']), np.mean(commits['ranking'])

In [6]:
colleges = ['alabama', 'georgia', 'ucla', 'usc']

for col in colleges:
    print(f'{col}: {getReciprocity(col)}, {getWeightedReciprocity(col)}, {getAvgRank(col)}')

alabama: 0.20161290322580644, 0.20485215309831903, (0.935922869955157, 0.9526760000000002)
georgia: 0.23255813953488372, 0.23332070653223722, (0.9266302631578944, 0.93007)
ucla: 0.2716049382716049, 0.2670706084623261, (0.9078557142857147, 0.8903636363636364)
usc: 0.06324110671936758, 0.06319227802069298, (0.9237865306122445, 0.9230499999999999)


In [7]:
rec = pd.DataFrame(colleges, columns=['colleges'])
rec['rec'] = rec.apply(lambda row : getReciprocity(row['colleges']), axis = 1)
rec['w_rec'] = rec.apply(lambda row : getWeightedReciprocity(row['colleges']), axis = 1)
rec['offer_rank'], rec['commit_rank']= zip(*rec.apply(lambda row : getAvgRank(row['colleges']), axis = 1))
rec

,colleges,rec,w_rec,offer_rank,commit_rank
0,alabama,0.201613,0.204852,0.935923,0.952676
1,georgia,0.232558,0.233321,0.926630,0.930070
2,ucla,0.271605,0.267071,0.907856,0.890364
3,usc,0.063241,0.063192,0.923787,0.923050


In [8]:
rec.to_csv('data/reciprocity.csv')

In [9]:
def getGraph(college):
    offers = pd.read_csv(f"data/offers/{college}_offers.csv")
    commits = pd.read_csv(f"data/commits/{college}_commits.csv")

    offers['col'] = college
    commits['col'] = college
    
    G_offer = nx.from_pandas_edgelist(offers, source = 'col', target = 'name', edge_attr='ranking', create_using = nx.DiGraph())
    G_commit = nx.from_pandas_edgelist(commits, source = 'name', target = 'col', edge_attr='ranking',create_using = nx.DiGraph())

    G = nx.compose(G_offer, G_commit)

    return G

In [10]:
for col in colleges:
    print(f'{col}: {nx.reciprocity(getGraph(col))}')

alabama: 0.20717131474103587
georgia: 0.23346303501945526
ucla: 0.2716049382716049
usc: 0.06976744186046512
